# Import necessary libraries

In [1]:
import numpy as np
import os
from imageio import imread
import datetime
import random as rn
from keras import backend as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

# Setting random seeds and reading csv files

In [2]:
# Set random seed for reproducibility
np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)  # tf.set_random_seed() is deprecated

In [3]:
# Read the training and validation CSV files
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32  # Set the batch size (experiment with this value)

# Generator

In [4]:

# Generator function
def generator(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.arange(0, 30)  # Use all 30 frames in each video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, len(img_idx), 100, 100, 3))  # Resize images to 100x100
            batch_labels = np.zeros((batch_size, 5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path + '/' + t[folder + (batch * batch_size)].split(';')[0])
                for idx, item in enumerate(img_idx):
                    image = imread(source_path + '/' + t[folder + (batch * batch_size)].strip().split(';')[0] + '/' + imgs[item]).astype(np.float32)
                    image_resized = np.resize(image, (100, 100, 3))  # Use np.resize instead of imresize
                    batch_data[folder, idx, :, :, :] = (image_resized / 255.0)
                batch_labels[folder, int(t[folder + (batch * batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

# Settings paths and parameters

In [5]:
# Paths and parameters
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20  # Choose the number of epochs
print('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


# Model creation

In [6]:
# Model creation
model = Sequential([
    Conv3D(32, kernel_size=(3, 3, 3), input_shape=(30, 100, 100, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling3D(pool_size=(2, 2, 2)),
    
    Conv3D(64, kernel_size=(3, 3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling3D(pool_size=(2, 2, 2)),
    
    Conv3D(128, kernel_size=(3, 3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling3D(pool_size=(2, 2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(5, activation='softmax')
])

c:\Users\shahp\anaconda3\envs\msc\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Model Summary

In [7]:
optimiser = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 30, 100, 100,   │         2,624 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 100, 100,   │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 30, 100, 100,   │             0 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 15, 50, 50, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 15, 50, 50, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 15, 50, 50, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 15, 50, 50, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 7, 25, 25, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 7, 25, 25, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 25, 25, 128) │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 7, 25, 25, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 3, 12, 12, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 55296)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    28,312,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,596,869 (109.09 MB)

 Trainable params: 28,595,397 (109.08 MB)

 Non-trainable params: 1,472 (5.75 KB)

None


In [8]:

# Create train and validation generators
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

# Define model checkpoint and learning rate reduction
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)

# Define the filepath with .keras extension
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

# Use ModelCheckpoint callback with filepath ending in .keras
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

# Define ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto', min_lr=0.00001)

# # After training, rename the saved model file to .h5
# import shutil
# shutil.move(filepath, filepath[:-6] + '.h5')

# Model Training

In [9]:
# Fit the model
history = model.fit(
    train_generator,
    steps_per_epoch=num_train_sequences // batch_size,
    epochs=num_epochs,
    verbose=1,
    callbacks=[checkpoint, LR],
    validation_data=val_generator,
    validation_steps=num_val_sequences // batch_size
)


C:\Users\shahp\AppData\Local\Temp\ipykernel_24224\1538011588.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path + '/' + t[folder + (batch * batch_size)].strip().split(';')[0] + '/' + imgs[item]).astype(np.float32)


Source path =  ./Project_data/train ; batch size = 32
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - categorical_accuracy: 0.2701 - loss: 2.1892Source path =  ./Project_data/val ; batch size = 32

Epoch 1: val_loss improved from inf to 2.49137, saving model to model_init_2024-05-2918_11_32.901132/model-00001-1.79750-0.33281-2.49137-0.21875.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 217s 10s/step - categorical_accuracy: 0.2731 - loss: 2.1706 - val_categorical_accuracy: 0.2188 - val_loss: 2.4914 - learning_rate: 0.0010
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - categorical_accuracy: 0.4480 - loss: 1.3553

# Saving Model

In [ ]:
from keras.models import load_model

# Identify the best model saved
best_model_path = max([os.path.join(model_name, f) for f in os.listdir(model_name)], key=os.path.getctime)

# Load the best model saved in .keras format
best_model = load_model(best_model_path)

# Save the best model in .h5 format
best_model.save(best_model_path.replace('.keras', '.h5'))
